In [1]:
import math, re, os, zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
from tensorflow.keras import layers
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

Tensorflow version 2.19.0


In [2]:
# Ruta del archivo ZIP
zip_path = 'C:/Users/Sebastian/Documents/GitHub/clasificacion_flores/data/raw/flower-classification-with-tpus.zip'

# Extraer el contenido
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('C:/Users/Sebastian/Documents/GitHub/clasificacion_flores/data/raw')

In [3]:
# ==== Configuración de variables ====

classes = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102

# tfrecords jpeg 224x224 path
ds_path = 'C:/Users/Sebastian/Documents/GitHub/clasificacion_flores/data/raw/tfrecords-jpeg-224x224'

train_files = tf.io.gfile.glob(f"{ds_path}/train/*.tfrec")
val_files = tf.io.gfile.glob(f"{ds_path}/val/*.tfrec")
test_files = tf.io.gfile.glob(f"{ds_path}/test/*.tfrec")    

In [4]:
# ==== Funciones para resultados y visualización ====

def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(classes)))
    ax.set_xticklabels(classes, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(classes)))
    ax.set_yticklabels(classes, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()
    plt.savefig('conf_mat.jpg')

def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])
    plt.savefig('training_curves.jpg')

In [5]:
# ==== Funciones para manejo de datos ====

AUTO = tf.data.experimental.AUTOTUNE
image_size = [224, 224]
batch_size = 32

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.keras.applications.efficientnet.preprocess_input(image)  # Normaliza para EfficientNet
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_saturation(image, 0.6, 1.6)
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    return image, label

def get_training_dataset():
    dataset = load_dataset(train_files, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(val_files, labeled=True, ordered=ordered)
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(test_files, labeled=False, ordered=ordered)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

num_training_images = count_data_items(train_files)
num_val_images = count_data_items(val_files)
num_test_images = count_data_items(test_files)
steps_per_epoch = num_training_images // batch_size
val_steps = -(-num_val_images // batch_size)
test_steps = -(-num_test_images // batch_size)
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(num_training_images, num_val_images, num_test_images))


train = get_training_dataset()
val = get_validation_dataset()
test = get_test_dataset()

Dataset: 12753 training images, 3712 validation images, 7382 unlabeled test images


In [6]:
# ==== Transferencia de aprendizaje ====

base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'  # GlobalAveragePooling2D
    )

base_model.trainable = False

# Añadir head
model = tf.keras.Sequential([
    base_model,
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(classes), activation='softmax')
    ])

# Compilar
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
    )

# Entrenar
history = model.fit(
    train,
    epochs=15,
    steps_per_epoch=steps_per_epoch,
    validation_data=val,
    validation_steps=val_steps,
)

# Visualización antes de fine-tuning
display_training_curves(
    history.history['sparse_categorical_accuracy'],
    history.history['val_sparse_categorical_accuracy'],
    'accuracy', 211
)
display_training_curves(
    history.history['loss'],
    history.history['val_loss'],
    'loss', 212
)

# Fine-tuning
base_model.trainable = True

for layer in base_model.layers[:-10]:  # Descongelar solo las últimas 10 capas
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy']
              )

history_finetune = model.fit(
    train,
    epochs=5,
    steps_per_epoch=steps_per_epoch,
    validation_data=val,
    validation_steps=val_steps
)

# Visualización despues de fine-tuning
display_training_curves(
    history_finetune.history['sparse_categorical_accuracy'],
    history_finetune.history['val_sparse_categorical_accuracy'],
    'accuracy (fine-tune)', 211
)
display_training_curves(
    history_finetune.history['loss'],
    history_finetune.history['val_loss'],
    'loss (fine-tune)', 212
)

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/15
 75/398 ━━━━━━━━━━━━━━━━━━━━ 5:48 1s/step - loss: 3.9058 - sparse_categorical_accuracy: 0.1920

KeyboardInterrupt: 

In [ ]:
# ==== Matriz de confusión ====

# Conjunto de validación en orden
cmdataset = get_validation_dataset(ordered=True)

# Separar imágenes y etiquetas
images_ds = cmdataset.map(lambda image, label: image)
labels_ds = cmdataset.map(lambda image, label: label).unbatch()

# Etiquetas reales
true_labels = next(iter(labels_ds.batch(num_val_images))).numpy()

# Predicciones
predicted_probabilities = model.predict(images_ds, steps=val_steps)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)

# Métricas
conf_matrix = confusion_matrix(true_labels, predicted_labels, labels=range(len(classes)))
f1 = f1_score(true_labels, predicted_labels, labels=range(len(classes)), average='macro')
precision = precision_score(true_labels, predicted_labels, labels=range(len(classes)), average='macro')
recall = recall_score(true_labels, predicted_labels, labels=range(len(classes)), average='macro')

# Normalizar la matriz de confusión
conf_matrix_normalized = (conf_matrix.T / conf_matrix.sum(axis=1)).T

# Aplicamos función para graficar
display_confusion_matrix(conf_matrix_normalized, f1, precision, recall)